# LLM 뉴스 요약

In [ ]:
from api_wrapper import ApiWrapper

api = ApiWrapper()

news_list = api.get_news_data()
news_list

In [ ]:
news_list["AAPL"]

In [ ]:
target = news_list["AAPL"]

name = "애플"

data = []
for i in target:
    data.append(i["title"].replace(":", "")  + " : " + i["description"].replace(":", ""))

data

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
from threading import Thread

model_name = "LGAI-EXAONE/EXAONE-Deep-7.8B"
streaming = False    # choose the streaming option

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


prompt = \
f"""
당신은 탁월한 증권사 애널리스트입니다.
당신은 지금 {name} 종목의 분석 보고서를 발간해야 합니다.

[요구 사항]
{name} 기업의 주요 헤드라인을 검토하고, 분석 보고서에 들어갈 시황, 주요 이슈를 2000자로 구성된 {name} 기업 분석 보고서를 만들어 주세요.
개인 투자자 입장에서 주의하거나 생각해봐야 할 시사점들이 있다면 분석 보고서에 반영해 주세요.
주요 헤드라인의 모든 내용이 {name} 기업과 연관되어 있지 않을 수도 있습니다. 이 경우 관련 내용을 담은 헤드라인만 보고서에 반영해 주세요.

[참고 사항]
주요 헤드라인은 한 줄의 '[기사 제목]: [description]' 으로 구성되어 있습니다.

[이 시각 {name} 기업의 주요 헤드라인]
{"\n".join(data)}

"""

messages = [
    {"role": "user", "content": prompt}
]
input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)

if streaming:
    streamer = TextIteratorStreamer(tokenizer)
    thread = Thread(target=model.generate, kwargs=dict(
        input_ids=input_ids.to("cuda"),
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=32768,
        do_sample=True,
        temperature=0.8,
        top_p=0.95,
        streamer=streamer
    ))
    thread.start()

    for text in streamer:
        print(text, end="", flush=True)
else:
    output = model.generate(
        input_ids.to("cuda"),
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=32768,
        do_sample=True,
        temperature=0.8,
        top_p=0.95,
    )
    print(tokenizer.decode(output[0]))

In [ ]:
result = tokenizer.decode(output[0]).split("</thought>")[1].replace("[|endofturn|]", "")

In [ ]:
prompt = \
f"""
당신은 탁월한 경제 연구원입니다.
당신은 지금 {name} 종목의 분석 보고서를 발간해야 합니다.

[요구 사항]
다음 [초안]을 보고, 세 문단으로 구성된 보고서 최종본을 만들어 주세요.

[제약 사항]
각각의 문단은 연속되고 완결된 문장으로 구성되어야 합니다.
별도의 소제목이나 구분 기호는 넣지 말아 주세요.
문장 간 연결은 자연스럽고 매끄러워야 합니다. 필요하다면 문단을 추가할 수 있습니다.

[초안]
{result}
"""

messages = [
    {"role": "user", "content": prompt}
]
input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)

if streaming:
    streamer = TextIteratorStreamer(tokenizer)
    thread = Thread(target=model.generate, kwargs=dict(
        input_ids=input_ids.to("cuda"),
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=32768,
        do_sample=True,
        temperature=0.8,
        top_p=0.95,
        streamer=streamer
    ))
    thread.start()

    for text in streamer:
        print(text, end="", flush=True)
else:
    output = model.generate(
        input_ids.to("cuda"),
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=32768,
        do_sample=True,
        temperature=0.8,
        top_p=0.95,
    )
    print(tokenizer.decode(output[0]))

In [ ]:
result = tokenizer.decode(output[0]).split("</thought>")[1].replace("[|endofturn|]", "")
print(result)